Importações

In [2]:
import time
import scipy.linalg as sp
import numpy as np
import sympy as sym

## ITEM 1

In [ ]:
x = [10, 10, 10, 10, 10]

In [3]:
# Test cell
a=[1, 2, 3, 4, 5]
a[-3]
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
f = x1**2 + x2
f.diff(x1).subs(x1, 1)

2

In [ ]:
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = .0005 # chosen
maxIterations = 100000 # so it doesnt loop forever in case of failure
def F(x):
    x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
    R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]
    f1 = x1*x2 +x1 -3*x5
    f2 = 2*x1*x2 +x1 +3*R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 -R[0]*x5
    f3 = 2*x2*(x3**2) +R[-4]*x2*x3 + 2*R[-6]*(x3**2) +R[-5]*x3 -8*x5
    f4 = R[-2]*x2*x4 +2*(x4**2) -4*R[0]*x5
    f5 = x1*x2 +x1 +R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 +R[-6]*(x3**2) +R[-5]*x3 +(x4**2) -1
    return [f1,f2,f3,f4,f5]

def J(x):
    return

### Método de Newton

In [ ]:
def methodNewton(n, Xo, tolerance, maxIterations):
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    k = 1
    while k <= maxIterations:
        FXo = -F(Xo)
        JXo = J(Xo)
        y = np.linalg.solve(JXo, FXo)
        x = x + y
        if np.linalg.norm(y) < tolerance:
            output = x
            break
        else:
            k += 1
        
    return output

### Método de Broyden

In [ ]:
Po = x # initial approximation
tolerance = .0005 # arbitrary
maxIterations = 100000 # so it doesnt loop forever in case of failure
def f(x):
    return
def df(x):
    return
def J(x):
    result = df(x)
    return result
def F(x):
    return


def methodBroyden(n, x, tolerance, maxIterations): # n is the number of equations and unknowns
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    Ao = J(x)
    v = F(x)
    A = np.transpose(Ao) #use gaussian elimination
    s = -A * v
    x = x + s
    k = 2
    
    while k <= maxIterations:
        w = v
        v = F(x)
        y = v - w
        z = -A* y
        p = -np.transpose(s) * z
        ut = np.transpose(np.transpose(s) * A)
        A = A + (1/p)*(s+z)*ut
        s = -A * v
        x = x + s
        if np.linalg.norm(s) < tolerance:
            output = x
            break
        
        k += 1
        
    return output
             
    

### Comparação dos resultados

## ITEM 2

### Método de Newton

In [ ]:
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = .0005 # chosen
maxIterations = 100000 # so it doesnt loop forever in case of failure

def f(x):
    return


def methodNewton(p0, p1, tolerance, maxIterations):
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    i = 1
    q0 = f(p0)
    q1 = f(p1)
    while i <= maxIterations:
        p = p1 - q1*((p1-p0)/(q1-q0))
       
        if tolerance < p - p1 < tolerance:
            output = p
            break
        else:
            i += 1
            p0 = p1
            q0 = q1
            p1 = p
            q1 = f(p)
        
    return output

### Método de Broyden

In [ ]:
Po = x # initial approximation
tolerance = .0005 # arbitrary
maxIterations = 100000 # so it doesnt loop forever in case of failure
def f(x):
    return
def df(x):
    return
def J(x):
    result = df(x)
    return result
def F(x):
    return


def methodBroyden(n, x, tolerance, maxIterations): # n is the number of equations and unknowns
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    Ao = J(x)
    v = F(x)
    A = np.transpose(Ao) #use gaussian elimination
    s = -A * v
    x = x + s
    k = 2
    
    while k <= maxIterations:
        w = v
        v = F(x)
        y = v - w
        z = -A* y
        p = -np.transpose(s) * z
        ut = np.transpose(np.transpose(s) * A)
        A = A + (1/p)*(s+z)*ut
        s = -A * v
        x = x + s
        if np.linalg.norm(s) < tolerance:
            output = x
            break
        
        k += 1
        
    return output
          

### Comparação dos resultados

## ITEM 3

#### Bibliografia e referências: 

R.L. Burden & J.D. Faires, Numerical Analysis, Capítulos 2 e 10